In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./submission_svm.csv
./starting-notebook-2.ipynb
./submission-2.csv
./starting-notebook.ipynb
./methods.py
./comp.py
./data/train.csv
./data/test.csv
./__pycache__/methods.cpython-310.pyc


In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv", index_col=[0])

In [3]:
len(train_df)

170889

In [4]:
test_df.shape

(113918, 30)

In [5]:
from sklearn.metrics import roc_auc_score

In [6]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# import time
# from sklearn.feature_selection import SequentialFeatureSelector

# for tol in [-1e-2, -1e-3, -1e-4]:
#     start = time()
#     feature_selector = SequentialFeatureSelector(
#         LogisticRegression(),
#         n_features_to_select="auto",
#         direction="backward",
#         scoring="roc_auc",
#         tol=tol,
#         n_jobs=2,
#     )
#     model = make_pipeline(
#         StandardScaler(), feature_selector, LogisticRegression())
#     model.fit(X_train, y_train)
#     end = time()
#     print(f"\ntol: {tol}")
#     print(f"Features selected: {X_train.columns[model[1].get_support()]}")
#     print(
#         f"ROC AUC score: {roc_auc_score(y_train, model.predict_proba(X_train)[:, 1]):.3f}")
#     print(f"Done in {end - start:.3f}s")

In [7]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import xgboost as xgb
import itertools
from tqdm.autonotebook import tqdm
from methods import xgboost, SupportVec

# train_df.fillna(0, inplace=True)
y = train_df.pop("Label").to_numpy()
X = train_df.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

# best_model = xgboost(X_train, y_train)
best_model = SupportVec(X_train, y_train)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
print("Test ROC AUC: ", auc)

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/tmp/ipykernel_4158710/1102223266.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=  38.9s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  40.2s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=  42.3s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  43.3s
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=  43.3s
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=  43.7s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=  44.3s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=  45.3s
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=  49.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time= 1.8min
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time= 1.8min
[CV] END ..................C

KeyboardInterrupt: 

In [ ]:
# test_df.fillna(0, inplace=True)
pred_test = best_model.predict_proba(test_df.to_numpy())[:, 1]
pred_test

In [ ]:
print(test_df.shape)

In [ ]:
submission = pd.DataFrame({'id':test_df.index, 'Label':pred_test})
submission

In [19]:
submission.set_index('id').to_csv("submission.csv")

In [ ]:
!head -5 submission.csv